In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is sheenu and I am a AI intern")])

AIMessage(content="Hello Sheenu! It's nice to meet you. \n\nI'm glad to hear you're an AI intern. That's exciting! What kind of projects are you working on?  \n\nI'm always interested in learning more about what other AI models are up to. 😊\n\n", response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 27, 'total_tokens': 92, 'completion_time': 0.133381486, 'prompt_time': 0.002615153, 'queue_time': None, 'total_time': 0.135996639}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-22ee5429-bbca-42bd-a21d-65f5d474e672-0', usage_metadata={'input_tokens': 27, 'output_tokens': 65, 'total_tokens': 92})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is sheenu and I am a AI intern"),
        AIMessage(content="Hello Sheenu! It's nice to meet you.  \n\nWelcome to the world of AI"),
        HumanMessage(content="what's my name and profession?")
    ]
)

AIMessage(content="You told me your name is Sheenu and that you are an AI intern.  \n\nIs there anything else you'd like to tell me about yourself? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 65, 'total_tokens': 103, 'completion_time': 0.077455245, 'prompt_time': 0.004399494, 'queue_time': None, 'total_time': 0.08185473900000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9d28f4fc-ea18-4f49-ab7e-e6a8cf39b3d6-0', usage_metadata={'input_tokens': 65, 'output_tokens': 38, 'total_tokens': 103})

### MESSAGE HISTORY
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [7]:
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(model,get_session_history)  #now we can able to interact with out model based on message history

this function will ensure that every session will store different chat history

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [9]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is sheenu and I am a Chief AI Engineer")],
    config=config
)

In [10]:
response

AIMessage(content="Hello Sheenu, it's nice to meet you!\n\nBeing a Chief AI Engineer is an exciting role. What kind of projects are you working on these days?  \n\nI'm eager to learn more about your work in the field of AI. Perhaps you could tell me about:\n\n* **The industry you work in:**  Are you in healthcare, finance, tech, or something else?\n* **The types of AI models you develop:**  Are you focused on machine learning, deep learning, natural language processing, computer vision, or a combination?\n* **Any recent challenges or successes you've encountered:** What are some of the most interesting things about your job?\n\n\n\nI'm here to listen and learn!  \n\n", response_metadata={'token_usage': {'completion_tokens': 155, 'prompt_tokens': 28, 'total_tokens': 183, 'completion_time': 0.323318752, 'prompt_time': 0.002750903, 'queue_time': None, 'total_time': 0.326069655}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}

In [11]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Sheenu. \n\nI remember that you introduced yourself at the beginning of our conversation! 😊  \n\n', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 196, 'total_tokens': 223, 'completion_time': 0.054680569, 'prompt_time': 0.012107423, 'queue_time': None, 'total_time': 0.066787992}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5cba03c8-8be5-4479-b7cf-779d813be53c-0', usage_metadata={'input_tokens': 196, 'output_tokens': 27, 'total_tokens': 223})

### prompt template
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [12]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [13]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is sheenu")]})

AIMessage(content="Hello Sheenu! It's nice to meet you. 😊  \n\nIs there anything I can help you with today?\n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 37, 'total_tokens': 65, 'completion_time': 0.056653792, 'prompt_time': 0.002960943, 'queue_time': None, 'total_time': 0.059614735}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d9d97b31-bd65-4f6e-8df4-f5d1b0bd1f64-0', usage_metadata={'input_tokens': 37, 'output_tokens': 28, 'total_tokens': 65})

In [14]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [15]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is sheenu")],
    config=config
)

response

AIMessage(content="Hi Sheenu, it's nice to meet you! 👋 \n\nIs there anything I can help you with today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 37, 'total_tokens': 68, 'completion_time': 0.062940628, 'prompt_time': 0.002924623, 'queue_time': None, 'total_time': 0.065865251}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1e63694d-b6cc-49a5-ad0d-e585905960a7-0', usage_metadata={'input_tokens': 37, 'output_tokens': 31, 'total_tokens': 68})

In [16]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Sheenu!  \n\nI remember that you told me. 😊  \n\n\nLet me know if you have any other questions.  \n'

In [17]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [18]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is sheenu")],"language":"hindi"})
response.content

'नमस्ते शीनू! 😊  \n\nआपको कैसे हो?  \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [19]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

input_messages_key="messages" parameter specifies the key in the input dictionary where the messages are stored. This key tells the function where to find the list of messages within the input data structure.

In [20]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'Namaste Krish! 😊  \n\nआप कैसे हैं?  \n'

In [21]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [22]:
response.content

'आपका नाम कृष्ण है।  \n'

### Managing the Conversation history